In [102]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/the-movies-dataset/ratings.csv
/kaggle/input/the-movies-dataset/links_small.csv
/kaggle/input/the-movies-dataset/credits.csv
/kaggle/input/the-movies-dataset/keywords.csv
/kaggle/input/the-movies-dataset/movies_metadata.csv
/kaggle/input/the-movies-dataset/ratings_small.csv
/kaggle/input/the-movies-dataset/links.csv


In [103]:
meta = pd.read_csv("/kaggle/input/the-movies-dataset/movies_metadata.csv")

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [104]:
meta = meta[["id","original_title","original_language","genres"]]

meta = meta.loc[meta["original_language"]=="en",:]  
#loc는 원하는 컬럼과 로우만 뽑아주는 함수
#,:는 전체를 뽑아오겠다는 걸 명시적으로 적어준것(없어도 됌) + ,이후 ["",""]이렇게 칼럼명 적으면 그거만 뽑아져나

In [105]:
meta = meta.rename(columns = {"id":"movieId","original_title":"title","original_language":"language"})
meta

,movieId,title,language,genres
0,862,Toy Story,en,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '..."
1,8844,Jumanji,en,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '..."
2,15602,Grumpier Old Men,en,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ..."
3,31357,Waiting to Exhale,en,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam..."
4,11862,Father of the Bride Part II,en,"[{'id': 35, 'name': 'Comedy'}]"
...,...,...,...,...
45459,222848,Caged Heat 3000,en,"[{'id': 878, 'name': 'Science Fiction'}]"
45460,30840,Robin Hood,en,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name..."
45463,67758,Betrayal,en,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam..."
45464,227506,Satana likuyushchiy,en,[]


In [106]:
meta.movieId = pd.to_numeric(meta.movieId)
#meta.movieId

In [107]:
#def str_to_set(x):
#    genre_set = set()
#    for item in eval(x):
#        genre_set.add(item['name'])
#    return genre_set

def str_to_set(x):
    return {x["name"] for x in eval(x)}

#s = meta["genres"][0]

#myset = set()
#for x in eval(s):
#    myset.add(x["name"])
#myset = {x["name"] for x in eval(s)} 랑 위 세줄이랑 같음!
#print(s)


In [108]:
meta.genres = meta.genres.apply(str_to_set)  #apply() 하면 각 값이 함수의 입력으로 들어가서 결과 값이 다시 들어옴

In [109]:
keywords = pd.read_csv("/kaggle/input/the-movies-dataset/keywords.csv")
keywords.keywords = keywords["keywords"].apply(str_to_set)

In [110]:
keywords = keywords.rename(columns = {"id":"movieId"})
keywords

,movieId,keywords
0,862,"{toy comes to life, jealousy, friends, friends..."
1,8844,"{giant insect, disappearance, new home, based ..."
2,15602,"{best friend, old men, duringcreditsstinger, f..."
3,31357,"{based on novel, interracial relationship, sin..."
4,11862,"{midlife crisis, pregnancy, mother daughter re..."
...,...,...
46414,439050,{tragic love}
46415,111109,"{pinoy, artist, play}"
46416,67758,{}
46417,227506,{}


In [111]:
meta = pd.merge(meta,keywords,on="movieId",how="inner") 
#inner 양쪽다 있는거 , outer 한쪽에만 있어도 되는거 (없으면 null)
#left는 왼쪽은 다넣고 오른쪽은 왼쪽에 있는것중 있는것만,right는 left 반대로
meta

,movieId,title,language,genres,keywords
0,862,Toy Story,en,"{Family, Animation, Comedy}","{toy comes to life, jealousy, friends, friends..."
1,8844,Jumanji,en,"{Family, Adventure, Fantasy}","{giant insect, disappearance, new home, based ..."
2,15602,Grumpier Old Men,en,"{Romance, Comedy}","{best friend, old men, duringcreditsstinger, f..."
3,31357,Waiting to Exhale,en,"{Romance, Drama, Comedy}","{based on novel, interracial relationship, sin..."
4,11862,Father of the Bride Part II,en,{Comedy},"{midlife crisis, pregnancy, mother daughter re..."
...,...,...,...,...,...
32847,222848,Caged Heat 3000,en,{Science Fiction},{}
32848,30840,Robin Hood,en,"{Action, Romance, Drama}",{}
32849,67758,Betrayal,en,"{Action, Drama, Thriller}",{}
32850,227506,Satana likuyushchiy,en,{},{}


In [112]:
dk = meta.loc[meta.title == "The Dark Knight"].iloc[0] #iloc은 loc인데 순서로 뽑는거
dkr = meta.loc[meta.title == "The Dark Knight Rises"].iloc[0]
pd.concat([dk,dkr],axis = 1).T #concat 데이터를 붙이는거, axis는 옆으로 붙혀라..?

,movieId,title,language,genres,keywords
10278,155,The Dark Knight,en,"{Action, Crime, Drama, Thriller}","{dc comics, organized crime, crime fighter, jo..."
14315,49026,The Dark Knight Rises,en,"{Action, Crime, Drama, Thriller}","{dc comics, time bomb, hostage drama, flood, c..."


In [113]:
def jaccard_similarity(a,b):
    if len(a|b) ==0:
        return 0
    return len(a&b)/len(a|b)

In [114]:
dk_gk = dk.genres | dk.keywords
dkr_gk = dkr.genres | dkr.keywords
jaccard_similarity(dk_gk,dkr_gk)

0.37142857142857144

In [115]:
meta

,movieId,title,language,genres,keywords
0,862,Toy Story,en,"{Family, Animation, Comedy}","{toy comes to life, jealousy, friends, friends..."
1,8844,Jumanji,en,"{Family, Adventure, Fantasy}","{giant insect, disappearance, new home, based ..."
2,15602,Grumpier Old Men,en,"{Romance, Comedy}","{best friend, old men, duringcreditsstinger, f..."
3,31357,Waiting to Exhale,en,"{Romance, Drama, Comedy}","{based on novel, interracial relationship, sin..."
4,11862,Father of the Bride Part II,en,{Comedy},"{midlife crisis, pregnancy, mother daughter re..."
...,...,...,...,...,...
32847,222848,Caged Heat 3000,en,{Science Fiction},{}
32848,30840,Robin Hood,en,"{Action, Romance, Drama}",{}
32849,67758,Betrayal,en,"{Action, Drama, Thriller}",{}
32850,227506,Satana likuyushchiy,en,{},{}


In [116]:
ratings = pd.read_csv("/kaggle/input/the-movies-dataset/ratings_small.csv")
ratings

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
...,...,...,...,...
99999,671,6268,2.5,1065579370
100000,671,6269,4.0,1065149201
100001,671,6365,4.0,1070940363
100002,671,6385,2.5,1070979663


In [117]:
ratings.movieId = pd.to_numeric(ratings.movieId)
ratings = pd.merge(ratings,meta[["movieId","title"]],on="movieId",how="inner")
ratings

,userId,movieId,rating,timestamp,title
0,1,1371,2.5,1260759135,Rocky III
1,4,1371,4.0,949810302,Rocky III
2,7,1371,3.0,851869160,Rocky III
3,19,1371,4.0,855193404,Rocky III
4,21,1371,3.0,853852263,Rocky III
...,...,...,...,...,...
33295,652,127728,5.0,1439586990,8:46
33296,652,129009,4.0,1442690827,Love Is a Ball
33297,653,2103,3.0,948161066,Solaris
33298,659,167,4.0,836137550,K-PAX


In [118]:
matrix = ratings.pivot_table(index="userId",columns="title",values="rating") #row = index
matrix

title,!Women Art Revolution,'Gator Bait,'Twas the Night Before Christmas,10 Items or Less,10 Things I Hate About You,"10,000 BC",11'09''01 - September 11,12 + 1,12 Angry Men,1408,...,Young and Innocent,Zaat,Zabriskie Point,Zapped Again!,Zardoz,Zodiac,eXistenZ,xXx,¡Three Amigos!,Мой сводный брат Франкенштейн
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
668,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
669,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [119]:
def pearson_similarity(u1,u2):  #평점 평균 구해서 빼주고 거기서 코사인 유사도 구하는 그거! 
    u1_c = u1 - u1.mean()
    u2_c = u2 - u2.mean()
    denom = np.sqrt(np.sum(u1_c**2)*np.sum(u2_c**2)) #cos유사  #sqrt는 루트씌우
    if denom!=0:
        return np.sum(u1_c*u2_c)/denom
    else:
        return 0

In [120]:
dk_rating = matrix["The Dark Knight"]
pk_rating = matrix["Prom Night"]
pearson_similarity(dk_rating,pk_rating)

0.773565934694095

In [127]:
def find_similar_movies(input_title,matrix,n,alpha):
    input_meta = meta.loc[meta["title"] == input_title].iloc[0]
    input_set = input_meta.genres | input_meta.keywords
    
    result = []
    
    for this_title in matrix.columns:
        if this_title == input_title:
            continue
        this_meta = meta.loc[meta["title"]==this_title].iloc[0]
        this_set = this_meta.genres | this_meta.keywords
        
        pearson = pearson_similarity(matrix[this_title],matrix[input_title])
        jaccard = jaccard_similarity(this_set,input_set)
        
        score = alpha * pearson + (1-alpha) * jaccard
        result.append((this_title,pearson,jaccard,score))
    result.sort(key = lambda r: r[3], reverse=True)
    
    return result[:n]

In [128]:
result = find_similar_movies("The Dark Knight",matrix,10,0.3)
pd.DataFrame(result,columns = ["title","pearson","jaccard","score"])

,title,pearson,jaccard,score
0,Wild Wild West,0.773566,0.032258,0.254650
1,Prom Night,0.773566,0.022222,0.247625
2,Batman Begins,0.005015,0.292683,0.206383
3,Yamakasi - Les samouraïs des temps modernes,0.377145,0.125000,0.200643
4,Blue Thunder,0.326617,0.133333,0.191318
5,Midnight in the Garden of Good and Evil,0.373841,0.111111,0.189930
6,Topaz,0.377145,0.103448,0.185557
7,Big Bad Mama,0.344649,0.107143,0.178395
8,Sneakers,0.415830,0.068966,0.173025
9,The Enforcer,0.326617,0.103448,0.170399
